# hashmap

hashmaps são coleções de chaves e valores.

não importa quantos itens tiver, sempre que precisar acessar uma chave, a complexidade será sempre O(1).  
sabendo que a maior parte dos hashmaps usam estruturas de dados que assemelham-se com arrays, ou são arrays mesmo. uma pergunta pode surgir no meio do caminho:

**como que as entradas são possíveis ser encontradas dentro do array em tempo constante?**

a mágica são as **hash functions**, um dos componentes principais de um hashmap.

a maior parte das implementações de hashmaps usam um (ou mais) array(s) para salvar em memória os dados do hashmap.  
sabendo disso, a função da **hash function** é de mapear as chaves para o memória em si que cada chave aponta.

<img src="public/hash_table.png" width="100%" />

o funcionamento é mais ou menos parecido com isso daqui:

```rust
// create a fresh hashmap; and access a random key
fn main() {
    let hm = HashMap::new();
    hm.get("keytobehashed");
}

// inside the hashmap implementation, we'll find the following behavior
fn get(&self, key: String) {
    let index = self.hash(key);
    return self.array[index];
}
```

sabendo disso a implementação da **hash functions** é muito importante.  
por que ela será responsável por controlar colisões dentro do hashmap, assim evitando dados de serem sobrescritos.

por exemplo uma implementação muito ingênua de uma **hash function** poderia ser algo parecido com isso.

```rust
fn hash(&self, key: String) {
    return key.len();
}
```

nesse caso, o índice retornado para as chaves: `"sp"` e `"rj"` seria igual ₋ já que ambos tem o comprimento igual a `2` que mapearia para o índice `self.array[1]`, logo um sobrescreveria o outro, o que fundamentalmente é uma colisão.

## load factor

_**load factor**_ ou **fator de carga** é a proporção entre espaço total e espaço utilizado em um array.

o **load factor** funciona como um limiar que vai desencadear o redimensionamento do armazenamento do hashmap.  
esse redimensionamento também chamado de **rehashing** é quando o limiar é atingido e o array em que todos os itens estão cresce em tamanho com base em um **growth factor**, que podemos assumir como `2` para simplificar — o que quer dizer que toda vez que se atingir este limiar, o array por baixo vai dobrar de tamanho, e todos os itens serão movidos para essa nova localidade em memória. 

esse mecanismo em conjunto com as **hash functions** trabalham em conjunto para minimizar a possiblidade de colisões dentro de um hashmap.

a ideia de implementar o _**mecanismo de resizing baseado no load factor**_ é que matematicamente, quanto mais o espaço disponível do hashmap é utilizado, naturalmente é maior a probabilidade de colisões.

o valor do _**load factor**_ geralmente gira em torno de `±0.7`, existem muitas pesquisas que apontam que o **load factor** ideal para a maioria dos casos gira em torno desse valor.

## colisões e open addressing

**colisões** dentro de um hashmap acontecem, quando a **hash function** produz o mesmo output para dois valores diferentes, fazendo que os mesmos sejam inseridos no mesmo espaço da memória.

por um certo tempo na história, para lidar com colisões a abordagem era de simplesmente jogar o item para o próximo slot disponível.  
só que isso traz um grande problema.

por que no melhor dos casos a complexidade temporal seria `O(1)` que é caso não haja nenhuma colisão e se pode inserir ou ler o determinado elemento.  
e no pior dos casos a complexidade temporal seria `O(n)`, caso haja uma colisão e o próximo slot não esteja disponível, seria disponível ir até o final do array para inserir o item.

e essa abordagem se chama **open addressing** e nesse caso com **linear probing** que é ir progressivamente aumentando em 1 até encontrar um slot:  
`(hash + 1) % size, (hash + 2) % size, (hash + 3) % size, (hash + 4) % size, ...`

mas existem outras formas de resolver isso ainda usando **open addressing**, o Rust é um ótimo exemplo disso, e usa **quadratic probing**.

> ```rust
> pub struct HashMap<K, V, S = RandomState> { /* private fields */ }
> ```
> 
> A hash map implemented with **quadratic probing** and SIMD lookup.
>
> Link: https://doc.rust-lang.org/std/collections/struct.HashMap.html

a ideia é que ao invés de seguir linearmente, se procura nos slots de intervalos quadráticos.

`(hash + 1^2) % size, (hash + 2^2) % size, (hash + 3^2) % size, (hash + 4^2) % size, (hash + 5^2) % size, ...`

## colisões e chaining

uma alternativa diferente e comumente usada para quando se acontece uma colisão, é trocar a estrutura de dados do slot, de um item único para uma subcoleção, como um **array**, **linked list** ou em outros casos até **trees** como em Java.

<img src="public/hash_map_chaining.png" width="100%" />

é importante notar que usando essa abordagem, quando acessa um item em que houve uma colisão, o acesso deixa de ser `O(1)` e se torna `O(n)`, aonde `N` é o tamanho da subcoleção.